In [ ]:
# Needed for functions
import numpy as np
import matplotlib.pyplot as plt
import qiskit
import random

# Import Qiskit classes
import qiskit 
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister, Aer, IBMQ, execute
from qiskit.providers.aer import noise

# Import measurement calibration functions
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, CompleteMeasFitter, MeasurementFilter)

In [ ]:
# Load account
provider = IBMQ.load_account()

In [ ]:
# Set backends
backend_1 = provider.get_backend('ibmq_rome')
simulator = qiskit.Aer.get_backend('qasm_simulator')

In [ ]:
# Generate the calibration circuits
qr = qiskit.QuantumRegister(2)
meas_calibs, state_labels = complete_meas_cal(qubit_list=[0,1], qr=qr, circlabel='mcal')

# Execute the calibration circuits without noise
backend = qiskit.Aer.get_backend('qasm_simulator')
job = qiskit.execute(meas_calibs, backend=backend, shots=1000)
cal_results = job.result()

# The calibration matrix without noise is the identity matrix
meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')

## Creation of the noise model

Set real_device to 1 and create the noise model from the real device, or set real_device to 0 and add personalized noise sources manually

In [ ]:
#Create a noise model from a real device or specified by user? (0=manual, 1=real)
real_device=1

#User-specified noise model
if real_device==0:
   noise_model = noise.NoiseModel()
   device = Aer.get_backend('qasm_simulator')
  
   for qi in range(2):
      read_err = noise.errors.readout_error.ReadoutError([[0.9, 0.1],[0.1,0.9]])
      noise_model.add_readout_error(read_err, [qi])
      properties = device.properties()
      coupling_map = device.configuration().coupling_map
      basis_gates = noise_model.basis_gates
# Noise model from real device
elif real_device==1:
   #Select backend to extract noise model from
   device = backend_1

   #Create noise model
   properties = device.properties()
   coupling_map = device.configuration().coupling_map
   noise_model = noise.device.basic_device_noise_model(properties)
   basis_gates = noise_model.basis_gates

# Execute the calibration circuits using the noise model
backend = qiskit.Aer.get_backend('qasm_simulator')
noisy_job = qiskit.execute(meas_calibs, backend=backend, shots=1000, noise_model=noise_model)
results = noisy_job.result()

# Calculate the calibration matrix
meas_fitter = CompleteMeasFitter(results, state_labels, circlabel='mcal')
meas_filter = meas_fitter.filter

# What is the measurement fidelity?
print("Average Measurement Fidelity: %f" % meas_fitter.readout_fidelity())

In [ ]:
# Function to write density matrix on a line
def line(rho):
    """Writes rho on a line"""
    rho_line=str(rho[0,0]).strip('(').strip(')')+' '+str(rho[0,1]).strip('(').strip(')')+' '+str(rho[0,2]).strip('(').strip(')')+' '+str(rho[0,3]).strip('(').strip(')')+' '+str(rho[1,0]).strip('(').strip(')')+' '+str(rho[1,1]).strip('(').strip(')')+' '+str(rho[1,2]).strip('(').strip(')')+' '+str(rho[1,3]).strip('(').strip(')')+' '+str(rho[2,0]).strip('(').strip(')')+' '+str(rho[2,1]).strip('(').strip(')')+' '+str(rho[2,2]).strip('(').strip(')')+' '+str(rho[2,3]).strip('(').strip(')')+' '+str(rho[3,0]).strip('(').strip(')')+' '+str(rho[3,1]).strip('(').strip(')')+' '+str(rho[3,2]).strip('(').strip(')')+' '+str(rho[3,3]).strip('(').strip(')')
    return rho_line

# Setting up the experiment

In [ ]:
# Use the real machine (noise_simulation=0) or use the simulator with noise simulation (noise_simulation=1)
noise_simulation = 0
# Choose the backend
backend = backend_1
# 13 states covering the first octant of the sphere in the VPC space (rand=0) or states generated with random angles (rand=1)
rand = 0
# Path to save data
path = ''
# Number of shots per circuit
shots=1000

In [ ]:
# Create quantum and classical registers
qr = QuantumRegister(2)
cr = ClassicalRegister(2)   

# j = Number of states on the sphere
# p = number of measurements of each state

# Random setting
if rand==1:
   j=1000
   p=1
  
# 13 defined states, with p measurements for each point
elif rand==0:
   j=13
   p=10

qr = QuantumRegister(2)
cr = ClassicalRegister(2)   
# Loop on the points to be calculated
for m in range(0,j):
   m=12-m
   # Loop on measurements to be made for each state, useful to compute errors from the dispersion
   for k in range(0,p):
      print("State "+str(13-m)+", measurement number "+str(k)+".")
      log=open(path+"Quantum_VPC_measurement_"+str(k)+"_state_"+str(m+1)+".txt","w")
      log_corr=open(path+"Quantum_VPC_measurement_mitigated_errors_"+str(k)+"_state_"+str(m+1)+".txt","w")
      if rand==1:
         theta=np.random.uniform(0, np.pi)
         alpha=np.random.uniform(0, np.pi)
      # Parameters giving the 13 combinations (V_A,P_A,C) distributed on the sphere
      elif rand==0:
         if m==0:
            alpha=np.pi/4
            theta=0
         if m==1:
            alpha=0.523599
            theta=0
         if m==2:
            alpha=0.261825
            theta=0
         if m==3:
            alpha=0
            theta=0
         if m==4:
            alpha=np.pi/4
            theta=0.9817+np.pi/2
         if m==5:
            alpha=0.561489
            theta=0.989+np.pi/2
         if m==6:
            alpha=0.361367
            theta=0.65+np.pi/2
         if m==7:
            alpha=0.261825
            theta=0+np.pi/2
         if m==8:
            alpha=np.pi/4
            theta=0.47+np.pi/2
         if m==9:
            alpha=0.659058
            theta=0.43+np.pi/2
         if m==10:
            alpha=0.561489
            theta=0.24+np.pi/2
         if m==11:
            alpha=0.523599
            theta=0+np.pi/2
         if m==12:
            alpha=np.pi/4
            theta=np.pi/2

      circuits = []

      S = np.array([[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]])
      rho = np.array([[0.0+0.0j, 0.0+0.0j, 0.0+0.0j, 0.0+0.0j], [0.0+0.0j, 0.0+0.0j, 0.0+0.0j, 0.0+0.0j], [0.0+0.0j, 0.0+0.0j, 0.0+0.0j, 0.0+0.0j], [0.0+0.0j, 0.0+0.0j, 0.0+0.0j, 0.0+0.0j]])
      
      for k in range(1,17):
               circuit=QuantumCircuit(qr,cr)
               circuit.u3(alpha*2,0,0,qr[0])
               circuit.cu3(theta*2,0,0,qr[0],qr[1])
               if k==2 or k==14:
                  circuit.h(qr[1])
               if k==3 or k==15:
                  circuit.sdg(qr[1])
                  circuit.h(qr[1])
               if k==5 or k==8:
                  circuit.h(qr[0])
               if k==6:
                  circuit.h(qr[0])
                  circuit.h(qr[1])
               if k==7:
                  circuit.h(qr[0])
                  circuit.sdg(qr[1])
                  circuit.h(qr[1])
               if k==9 or k==12:
                  circuit.sdg(qr[0])
                  circuit.h(qr[0])
               if k==10:
                  circuit.sdg(qr[0])
                  circuit.h(qr[0])   
                  circuit.h(qr[1])
               if k==11:
                  circuit.sdg(qr[0])
                  circuit.h(qr[0])
                  circuit.sdg(qr[1])
                  circuit.h(qr[1])

               circuit.measure(qr[0], cr[0])
               circuit.measure(qr[1], cr[1])
               circuits.append(circuit)
      
      if noise_simulation==0:
          job = execute(circuits, backend=backend, shots=shots)
      elif noise_simulation==1: 
          job = execute(circuits, simulator, noise_model=noise_model,coupling_map=coupling_map,basis_gates=basis_gates, shots=shots)
      for correction in range(0,2):
          if correction==1:
                result=meas_filter.apply(job.result())
          elif correction==0:
                result = job.result()

          for i in range(1,17):
             res=result.get_counts(circuits[i-1])

             counts=np.array([0,0,0,0])
             for state, events in res.items():
                if float(state)==0:
                      counts[0]=float(events)
                if float(state)==10:
                      counts[1]=float(events)
                if float(state)==1:
                      counts[2]=float(events)
                if float(state)==11:
                   counts[3]=float(events)

             counts=counts/shots

             if i==1:
                S[0][0]=float(counts[0])+float(counts[1])+float(counts[2])+float(counts[3])
             elif i==2:
                S[0][1]=float(counts[0])-float(counts[1])+float(counts[2])-float(counts[3])
             elif i==3:
                S[0][2]=float(counts[0])-float(counts[1])+float(counts[2])-float(counts[3])
             elif i==4:
                S[0][3]=float(counts[0])-float(counts[1])+float(counts[2])-float(counts[3])
             elif i==5:
                S[1][0]=float(counts[0])+float(counts[1])-float(counts[2])-float(counts[3])
             elif i==9:
                S[2][0]=float(counts[0])+float(counts[1])-float(counts[2])-float(counts[3])
             elif i==13:
                S[3][0]=float(counts[0])+float(counts[1])-float(counts[2])-float(counts[3])
             elif i==6:
                S[1][1]=float(counts[0])-float(counts[1])-float(counts[2])+float(counts[3])
             elif i==7:
                S[1][2]=float(counts[0])-float(counts[1])-float(counts[2])+float(counts[3])
             elif i==8:
                S[1][3]=float(counts[0])-float(counts[1])-float(counts[2])+float(counts[3])
             elif i==10:
                S[2][1]=float(counts[0])-float(counts[1])-float(counts[2])+float(counts[3])
             elif i==11:
                S[2][2]=float(counts[0])-float(counts[1])-float(counts[2])+float(counts[3])
             elif i==12:
                S[2][3]=float(counts[0])-float(counts[1])-float(counts[2])+float(counts[3])
             elif i==14:
                S[3][1]=float(counts[0])-float(counts[1])-float(counts[2])+float(counts[3])
             elif i==15:
                S[3][2]=float(counts[0])-float(counts[1])-float(counts[2])+float(counts[3])
             elif i==16:
                S[3][3]=float(counts[0])-float(counts[1])-float(counts[2])+float(counts[3])

          rho[0][0]=float(S[0][3])+float(S[3][0])+float(S[3][3])+float(S[0][0])
          rho[0][1]=float(S[0][1])-1j*float(S[0][2])+float(S[3][1])-1j*float(S[3][2])
          rho[0][2]=float(S[1][0])+float(S[1][3])-1j*float(S[2][0])-1j*float(S[2][3])
          rho[0][3]=float(S[1][1])-1j*float(S[1][2])-1j*float(S[2][1])-float(S[2][2])
          rho[1][0]=float(S[0][1])+1j*float(S[0][2])+float(S[3][1])+1j*float(S[3][2])
          rho[1][1]=-float(S[0][3])+float(S[3][0])-float(S[3][3])+float(S[0][0])
          rho[1][2]=float(S[1][1])+1j*float(S[1][2])-1j*float(S[2][1])+float(S[2][2])
          rho[1][3]=float(S[1][0])-float(S[1][3])-1j*float(S[2][0])+1j*float(S[2][3])
          rho[2][0]=float(S[1][0])+float(S[1][3])+1j*float(S[2][0])+1j*float(S[2][3])
          rho[2][1]=float(S[1][1])-1j*float(S[1][2])+1j*float(S[2][1])+float(S[2][2])
          rho[2][2]=float(S[0][3])-float(S[3][0])-float(S[3][3])+float(S[0][0])
          rho[2][3]=float(S[0][1])-1j*float(S[0][2])-float(S[3][1])+1j*float(S[3][2])
          rho[3][0]=float(S[1][1])+1j*float(S[1][2])+1j*float(S[2][1])-float(S[2][2])
          rho[3][1]=float(S[1][0])-float(S[1][3])+1j*float(S[2][0])-1j*float(S[2][3])
          rho[3][2]=float(S[0][1])+1j*float(S[0][2])-float(S[3][1])-1j*float(S[3][2])
          rho[3][3]=-float(S[0][3])-float(S[3][0])+float(S[3][3])+float(S[0][0])
          rho=rho/4

          rho_line = line(rho)

          # Compute partial traces
          n1, n2, n=2, 2, 4
          rho_tensor=rho.reshape([n1, n2, n1, n2])
          rho_a=np.trace(rho_tensor, axis1=1, axis2=3)
          rho_b=np.trace(rho_tensor, axis1=0, axis2=2)

          # Compute V_k, P_k (note that V1a = V1b = V_A because the density matrix is hermitic. Also, we note P1 = P_A, P2 = P_B)
          V1a=2*abs(rho_a[0][1])
          V2a=2*abs(rho_b[0][1])
          V1b=2*abs(rho_a[1][0])
          V2b=2*abs(rho_b[1][0])
          P1=abs(rho_a[1][1]-rho_a[0][0])
          P2=abs(rho_b[1][1]-rho_b[0][0])

          # Compute C
          Sigma = np.array([[0, 0, 0, -1], [0, 0, 1, 0], [0, 1, 0, 0], [-1, 0, 0, 0]])
          rho_transpose = np.transpose(rho)
          V_p_rho=np.linalg.eigvals(rho)
          R = (rho.dot(Sigma)).dot(rho_transpose.dot(Sigma))
          V_p=np.linalg.eigvals(R)
          V_p.sort()
          arg1=0
          arg2=np.sqrt(float(abs(V_p[3])))-np.sqrt(float(abs(V_p[2])))-np.sqrt(float(abs(V_p[1])))-np.sqrt(float(abs(V_p[0])))
          C=max(arg1,arg2)

          # Output string data
          string = str(alpha) + ' ' +str(theta) + ' ' +str(V1a) + ' ' +str(P1)+ ' '+str(V2a)+' '+str(P2)+' '+str(C)+ ' '+rho_line
          if correction==0:
              log.write(string)
              log.close()
          if correction==1:
              log_corr.write(string)
              log_corr.close()